In [47]:
import numpy as np
import pandas as pd

In [110]:
data = pd.read_csv('hw4_data.csv')

data = data.set_index('date')

rts = data.pct_change().dropna()

#### sample

In [112]:
mean = rts.mean()
std = rts.std()
cov = (1/rts.shape[0]) * ((rts - mean).T @ (rts - mean))

#### annualized

In [111]:
n = 252

In [268]:
mu = (mean + 1)**n-1
var_a = (std**2 + (mean+1)**2)**n - (mean+1)**(2*n)
std_a = var_a**(1/2)
sigma = std_a * rts.corr() * std_a

In [270]:
rts.corr()

,gnk,bbby,hcci,skm,cs,azn,nvda,epac
gnk,1.000000,0.143774,0.335126,0.154859,0.398503,0.194425,0.267417,0.366859
bbby,0.143774,1.000000,0.228128,0.140678,0.260854,0.072005,0.223917,0.269887
hcci,0.335126,0.228128,1.000000,0.223341,0.514990,0.236256,0.321087,0.497233
skm,0.154859,0.140678,0.223341,1.000000,0.326674,0.199538,0.238067,0.262061
cs,0.398503,0.260854,0.514990,0.326674,1.000000,0.289900,0.441271,0.536134
azn,0.194425,0.072005,0.236256,0.199538,0.289900,1.000000,0.318483,0.234022
nvda,0.267417,0.223917,0.321087,0.238067,0.441271,0.318483,1.000000,0.368772
epac,0.366859,0.269887,0.497233,0.262061,0.536134,0.234022,0.368772,1.000000


In [273]:
std_a

gnk     0.830681
bbby    1.610613
hcci    0.642194
skm     0.298325
cs      0.316412
azn     0.304551
nvda    0.747883
epac    0.398939
dtype: float64

In [275]:
0.830681*1.610613*0.143774

0.19235604224368763

In [250]:
#### inputs

In [172]:
ones = np.ones(std_a.shape)

In [173]:
sigma_inv = np.linalg.inv(sigma)

#### global minimum variance

In [174]:
w_gm = (sigma_inv @ ones) / (ones.T @ sigma_inv @ ones)

In [175]:
mu_gm = w_gm @ mu

In [225]:
var_gm = w_gm.T @ sigma @ w_gm

In [226]:
np.sqrt(var_gm)

0.25961373077920297

#### target return

In [ ]:
def mv_var(mu_target):
    psi = (ones.T @ sigma_inv @ ones) * (mu.T @ sigma_inv @ mu) - (ones.T @ sigma_inv @ mu) ** 2
    sigma_t = var_gm + ((mu_target - mu_gm) ** 2) / (psi * var_gm)
    return sigma_t

In [ ]:
def mv_portfolio(mu_target):
    a = mu.T @ sigma_inv @ mu
    b = mu.T @ sigma_inv @ ones
    c = ones.T @ sigma_inv @ ones
    #
    w_mu = (1/b) * sigma_inv @ mu
    #
    l = (b*c*mu_target - (b**2)) / (a*c - (b**2))
    #
    w = l*w_mu + (1-l)*w_gm
    #
    return pd.Series(index=mu.index, data=w)

In [241]:
w = mv_portfolio(0.12)

In [242]:
w @ mu

0.12000000000000002

In [243]:
w.T @ sigma @ w

0.05981116349811591

In [247]:
mv_var(0.25)

0.4077888819145278

In [210]:
np.sum(w)

0.8740386249459117

In [216]:
frontier = pd.Series(data=[0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2])

In [218]:
frontier

0    0.04
1    0.06
2    0.08
3    0.10
4    0.12
5    0.14
6    0.16
7    0.18
8    0.20
dtype: float64

In [220]:
wts = frontier.apply(mv_portfolio)
wts.index = frontier
wts

,gnk,bbby,hcci,skm,cs,azn,nvda,epac
0.04,0.041475,0.015375,0.034328,0.474098,0.081335,0.397367,0.016549,0.084120
0.06,0.045407,0.016157,0.040893,0.454137,0.017406,0.404652,0.025044,0.073298
0.08,0.049339,0.016938,0.047458,0.434177,-0.046524,0.411937,0.033540,0.062477
0.10,0.053271,0.017720,0.054023,0.414217,-0.110453,0.419222,0.042036,0.051655
0.12,0.057204,0.018501,0.060587,0.394256,-0.174382,0.426507,0.050531,0.040833
0.14,0.061136,0.019283,0.067152,0.374296,-0.238311,0.433792,0.059027,0.030012
0.16,0.065068,0.020064,0.073717,0.354336,-0.302240,0.441077,0.067523,0.019190
0.18,0.069000,0.020846,0.080282,0.334375,-0.366169,0.448362,0.076018,0.008368
0.20,0.072933,0.021627,0.086847,0.314415,-0.430098,0.455646,0.084514,-0.002453


In [222]:
wts.apply(lambda x: x.T @ sigma @ x, axis=1)

0.04    0.078337
0.06    0.072926
0.08    0.068034
0.10    0.063662
0.12    0.059811
0.14    0.056480
0.16    0.053669
0.18    0.051378
0.20    0.049607
dtype: float64

In [223]:
wts.sum()

gnk     0.514833
bbby    0.166513
hcci    0.545286
skm     3.548308
cs     -1.569435
azn     3.838560
nvda    0.454782
epac    0.367500
dtype: float64